In [1]:
# read csv into df
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from constants import *
from glob import glob
from datetime import datetime
import os


print(DATA_GLOB_PLANINSKE_MAKRO_ALL)

df = pd.DataFrame(columns=['datum', "location_id", "pretocnost"])

location_dict = {
    "kum": 0,
    "lovrenška_jezera": 1,
    "osp": 2,
    "storžič": 3,
    "triglavski_narodni_park": 4,
    "vršič": 5,
}

for path in glob(DATA_GLOB_PLANINSKE_MAKRO_ALL):
    print(path)

    # get file name without extension and path
    file_name = os.path.basename(path).split('.')[0].lower().replace(" ", "_")
    if file_name == "zdruzeno":
        continue
    
    # file_name = path.split('/')[-1].split('.')[0].replace(" ", "_").lower()
    assert file_name in location_dict.keys(), f"file_name = {file_name}"
    print(file_name)
    location_id = location_dict[file_name]

    df_planine = pd.read_csv(path, sep=',', encoding='UTF-8')
    assert df_planine.columns[0] == 'datum', f"columns[0] = {df_planine.columns[0]}"
    assert df_planine.columns[1] == 'vhodi', f"columns[1] = {df_planine.columns[1]}"
    assert df_planine.columns[2] == 'izhodi', f"columns[2] = {df_planine.columns[2]}"
    # print(df_planine.head())
    
    # split "datum" by space
    # df_planine['datum'] = df_planine['datum'].str.split(' ')
    # df_planine['datum_dan'] = df_planine['datum'].str[0]
    # # df_planine['datum_ura'] = df_planine['datum'].str[1]
    # # drop "datum" column
    # df_planine = df_planine.drop(columns=['datum'])
    # # print(df_planine.head())

    
    # sum vhodi and izhodi
    df_planine['pretocnost'] = df_planine['vhodi'] + df_planine['izhodi']
    # drop "vhodi" and "izhodi" columns
    df_planine = df_planine.drop(columns=['vhodi', 'izhodi'])

    # add "location_id" column
    df_planine['location_id'] = location_id

    df = pd.concat([df, df_planine], ignore_index=True)

    # sum vhodi and izhodi based on "datum_dan"
    # df_planine = df_planine.groupby('datum').sum()
    print(df_planine.head())

    
    # break

df = df.sort_values(by=['datum', 'location_id'])
print(df.head())

df.to_csv(OUT_PLANINSTVO, index=False)



../data/planinstvo/2024/*.csv
../data/planinstvo/2024\Kum.csv
kum
        datum  pretocnost  location_id
0  2022-06-20           3            0
1  2022-06-21          24            0
2  2022-06-22           3            0
3  2022-06-23           5            0
4  2022-06-24           8            0
../data/planinstvo/2024\Lovrenška jezera.csv
lovrenška_jezera
        datum  pretocnost  location_id
0  2022-06-27         318            1
1  2022-06-28          66            1
2  2022-06-29         214            1
3  2022-06-30         288            1
4  2022-07-01         379            1
../data/planinstvo/2024\Osp.csv
osp
        datum  pretocnost  location_id
0  2022-06-15          46            2
1  2022-06-16          60            2
2  2022-06-17          79            2
3  2022-06-18          62            2
4  2022-06-19          40            2
../data/planinstvo/2024\Storžič.csv
storžič
        datum  pretocnost  location_id
0  2022-07-19          72            3
1  2022-07-2

In [ ]:
DATA_GLOB_PLANINSKE_MAKRO_ALL